# All Games Collecting
* Get the schedule from http://cba.sports.sina.com.cn/cba/schedule/all/?qleagueid=205&qmonth=&qteamid=
* Get all the links
* Loop through all the game links and scrape detailed stats

In [1]:
import requests
from bs4 import BeautifulSoup
# import lxml.html as lh
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import numpy as np
import re
import time


def get_page_content(url,encoding='UTF-8',remove_linebreaks=True
                    ,header = {'User-Agent': r'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                            r'Chrome/41.0.2227.1 Safari/537.36'}):
    session = requests.Session()
    base_url = url
    response = session.get(base_url, headers=header)
    response.encoding = encoding
    page_content = BeautifulSoup(response.content, "html.parser")
    page_content_str = str(page_content)
    if remove_linebreaks:
        page_content_str = re.sub('\n','',page_content_str)
    return page_content_str

### To do:
* get all values in "select" tag, just in case this base url does not work anymore
* get all links
* after get all links, scrape each game's stats
* 暂定 主队_link，统计_link这样的header, 后期可能单独建一个表放所有entity（球队，球员）的link
* automatically determine number of columns in the table
* Not sure if I want to rescrape all the data every week or just go look for the missing game links using known data
* 单节得分，教练，领队可以记在game表里

In [3]:
base_url = r'http://cba.sports.sina.com.cn/cba/schedule/all/?qleagueid=205&qmonth=&qteamid='

In [4]:
page_content = get_page_content(url=base_url,remove_linebreaks=True)
page_content


'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head><meta content="text/html; charset=utf-8" http-equiv="Content-type"/><title>CBA联赛赛程赛果_中国篮球数据库_篮球-CBA_新浪体育_新浪网</title><meta content="CBA联赛赛程赛果，CBA联赛直播，CBA联赛赛程赛果，CBA联赛直播，中国篮球数据库" name="keywords"/><meta content="新浪CBA联赛，提供CBA联赛最新赛程赛果、CBA联赛直播、CBA联赛总决赛、CBA联赛录像、CBA联赛视频集锦、CBA联赛球队球员、CBA联赛十佳球、CBA联赛快讯、CBA联赛图片等丰富的CBA联赛新闻和数据。" name="description"/><link href="http://www.sinaimg.cn/ty/deco/2013/0124/cbacss/css/base.css" rel="stylesheet" type="text/css"/><link href="http://www.sinaimg.cn/ty/deco/2013/0124/cbacss/css/procedure.css" rel="stylesheet" type="text/css"/><script src="http://www.sinaimg.cn/ty/deco/2013/0124/cbacss/css/cba.js" type="text/javascript"></script><script type="text/javascript">window.onload = function(){\tvar s_select_01 = new DivSelect("league");\tvar s_select_02 = new DivSelect("month");\tvar s_select_03 = new D

In [7]:
# Get table division
table_content_str = re.findall('<div class="blk_wrap"><table>(.*)</table></div>',str(page_content))

len(table_content)

1

### Get table headers

In [9]:
# Get headers


table_headers = re.findall('<th>(.*?)</th>',str(table_content))
table_headers

['轮次', '日期', '主队', '比分', '客队', '战报', '统计', '组图', '地点', '电视']

### Get table rows (table body, 也就是赛程表)

In [10]:
# Get table data



table_body = re.findall('<tbody>(.*?)</tbody>',str(table_content))
len(table_body)

1

In [11]:
table_body_str = table_body[0]
# get rid of invalid character
table_body_str = table_body_str.replace(u'\xa0',' ')

table_cells = re.findall('<td>(.*?)</td>',table_body_str)

# get rid of spaces
table_cells = list(map(lambda x: x.strip(),table_cells))


### Get actual texts and links

In [21]:
# get actual text
table_cells_txt = list(map(lambda x: re.findall('<a.*?>(.*)</a>',x)[0].strip() if '<a' in x else x, table_cells))
# get all the links
table_cells_links = list(map(lambda x: re.findall('<a href="(.*?)" target="_blank">'
                                                  ,x)[0].strip() if '<a' in x else None, table_cells))

4600
['第1轮', '2019-11-01 19:35', '广东', '107:98', '辽宁', '战报', '统计', '组图', '东莞', '', '第1轮', '2019-11-02 19:35', '江苏', '96:97', '浙江', '战报', '统计', '组图', '苏州', '']
4600
[None, None, 'http://cba.sports.sina.com.cn/cba/team/show/1/', 'http://cba.sports.sina.com.cn/cba/schedule/show/18381/', 'http://cba.sports.sina.com.cn/cba/team/show/9/', 'http://sports.sina.com.cn/basketball/cba/2019-11-01/doc-iicezzrr6602072.shtml', 'http://cba.sports.sina.com.cn/cba/schedule/show/18381/', 'http://slide.sports.sina.com.cn/cba/slide_2_792_233270.html', None, None, None, None, 'http://cba.sports.sina.com.cn/cba/team/show/2/', 'http://cba.sports.sina.com.cn/cba/schedule/show/18382/', 'http://cba.sports.sina.com.cn/cba/team/show/6/', 'http://sports.sina.com.cn/basketball/cba/2019-11-02/doc-iicezuev6752945.shtml', 'http://cba.sports.sina.com.cn/cba/schedule/show/18382/', 'http://slide.sports.sina.com.cn/cba/slide_2_786_233393.html', None, None]


### Save data as schedule table

In [22]:

import numpy as np
table_cells_txt = np.reshape(table_cells_txt,[-1,10]).tolist()
table_cells_links = np.reshape(table_cells_links,[-1,10]).tolist()

[['第1轮', '2019-11-01 19:35', '广东', '107:98', '辽宁', '战报', '统计', '组图', '东莞', ''], ['第1轮', '2019-11-02 19:35', '江苏', '96:97', '浙江', '战报', '统计', '组图', '苏州', ''], ['第1轮', '2019-11-02 19:35', '四川', '87:134', '新疆', '战报', '统计', '组图', '成都', ''], ['第1轮', '2019-11-02 19:35', '山东', '108:95', '八一', '战报', '统计', '组图', '济南', ''], ['第1轮', '2019-11-02 19:35', '北控', '108:103', '深圳', '战报', '统计', '组图', '北京', ''], ['第1轮', '2019-11-02 19:35', '青岛', '103:105', '吉林', '战报', '统计', '组图', '青岛', ''], ['第1轮', '2019-11-02 19:35', '广厦', '93:90', '福建', '战报', '统计', '组图', '诸暨', ''], ['第1轮', '2019-11-02 20:00', '上海', '93:84', '同曦', '战报', '统计', '组图', '上海', ''], ['第1轮', '2019-11-03 19:35', '天津', '81:103', '北京', '战报', '统计', '组图', '天津', ''], ['第1轮', '2019-11-03 19:35', '广州', '96:123', '山西', '战报', '统计', '组图', '广州', '']]
[[None, None, 'http://cba.sports.sina.com.cn/cba/team/show/1/', 'http://cba.sports.sina.com.cn/cba/schedule/show/18381/', 'http://cba.sports.sina.com.cn/cba/team/show/9/', 'http://sports.sina.com.cn/basketball/

In [23]:
df_schedule_text = pd.DataFrame(data = table_cells_txt,columns=table_headers)
df_schedule_links = pd.DataFrame(data = table_cells_links,columns=[header+'_link' for header in table_headers])

In [27]:
df_schedule_full = pd.merge(df_schedule_text,df_schedule_links,left_index=True,right_index=True)
# remove columns that are all None
df_schedule_full.dropna(axis=1,how='all',inplace=True)

In [28]:
# id consists of season and a 5-digit label of the game - 2019-2020 season, game No.1 = 2019202000001
# lable is generated by unique http address of the game stats
uid_column = ['20192020'+f'{item+1:05d}' for item in pd.factorize(df_schedule_full['统计_link'])[0].tolist()]
df_schedule_full['UID'] = uid_column
# get rid of Hanzi and convert into integers
df_schedule_full['轮次'] = df_schedule_full['轮次'].apply(lambda x: int(re.findall('(\d+)',x)[0]))

In [31]:
df_schedule_full.to_csv(f'../StatsData/GamesSchedulePage_{datetime.date.today()}.csv', index=False)

## Details Page

###  每节比分对比

In [23]:
hometeam_name = re.findall('<span class="team_name team_nameA" style="font-size:15px">(.*?)</span>'
                            ,page_content_str)[0]
awayteam_name = re.findall('<span class="team_name team_nameB" style="font-size:15px">(.*?)</span>'
                            ,page_content_str)[0]
hometeam_score = re.findall('<div class="team_score team_scoreA"><span>(\d*?)</span></div>',page_content_str)[0]
awayteam_score = re.findall('<div class="team_score team_scoreB"><span>(\d*?)</span></div>',page_content_str)[0]
print(hometeam_name,hometeam_score)
print(awayteam_name,awayteam_score)

广东 107
辽宁 98


In [24]:
# <div class="score_pad"><div class="scores"></div>
pts_qt_headers = re.findall('<div class="scores">(.*?)<div class="score_tab">',page_content_str)
pts_qt_headers_str = pts_qt_headers[0]

pts_qt_headers_lst = re.findall('<span.*?>(.*?)</span>',pts_qt_headers_str)
pts_qt_headers_lst = pts_qt_headers_lst[1:-1]
pts_qt_headers_lst

['第一节', '第二节', '第三节', '第四节', '加时一', '加时二', '加时三']

In [25]:
# <div class="score_pad"><div class="scores"></div>
pts_qt_pts = re.findall('<div class="team_score team_scoreA">.*?</div>(.*?)<div class="team_score team_scoreB">'
                        ,page_content_str)
pts_qt_pts_str = pts_qt_pts[0]
pts_qt_pts_str = pts_qt_pts_str.replace('\xa0','')
pts_qt_pts_lst = re.findall('<span.*?>(.*?)</span>',pts_qt_pts_str)
pts_qt_pts_lst = [int(x) if x else None for x in pts_qt_pts_lst]

In [26]:
hometeam_pts_qtr = pts_qt_pts_lst[0::2]
awayteam_pts_qtr = pts_qt_pts_lst[1::2]

print(hometeam_name,hometeam_score,hometeam_pts_qtr)
print(awayteam_name,awayteam_score,awayteam_pts_qtr)

广东 107 [23, 27, 32, 25, None, None, None]
辽宁 98 [22, 25, 21, 30, None, None, None]


###  主队统计数据

### Todo:
* 从game_schedule表内加上队伍名称，比赛ID
* save as json?? df_schedule_full[:3].to_json(force_ascii=False,orient='index')

In [36]:
def get_team_details(page_content_str,re_pattern):
    team_details = re.findall(re_pattern,page_content_str)[0]
    return team_details

def get_table_coach(team_details):
    coach = re.findall('主教练：(.*?)领队',team_details)[0].replace('\xa0','')
    return coach

def get_table_lingdui(team_details):
    lingdui = re.findall('领队：(.*?)<',team_details)[0].replace('\xa0','')
    return lingdui

def get_table_headers(team_details):
    team_table_str = re.findall('<table>(.*?)</table>',team_details)[0]
    team_table_header_str = re.findall('<thead>(.*?)</thead>',team_table_str)[0]
    team_table_header_lst = re.findall('<th>(.*?)</th>',team_table_header_str)
    return team_table_header_lst

def clean_cells(cell_data):
    if '<a' in cell_data:
        cell_data = re.findall('<a.*?>(.*?)</a>',cell_data)[0]
    if 'document.write' in cell_data:
        try:
            cell_data = re.findall('[(]"(\d*)"[)]',cell_data)[0]
        except IndexError:
            cell_data = re.findall('[(]"(\d*/\d)"[)]',cell_data)[0]
    cell_data = re.sub('\s','', cell_data)
    cell_data = re.sub('是','1', cell_data)
    cell_data = re.sub('否','0', cell_data)
    return cell_data

def get_table_cells(team_details,headers_count):
    team_table_str = re.findall('<table>(.*?)</table>',team_details)[0]
    team_table_cells_str = re.findall('<tbody style="">(.*?)</tbody>',team_table_str)[0]
    team_table_cells_lst = re.findall('<td>(.*?)</td>',team_table_cells_str)
    team_table_cells_lst = list(map(clean_cells,team_table_cells_lst))
    team_table_cells_lst = np.reshape(team_table_cells_lst,(-1,headers_count)).tolist()
    return team_table_cells_lst

In [37]:
def get_hometeam_stats(page_content_str,re_pattern=
                       '<div class="part part01 blk">(.*?)<div class="part part02 blk">'):
    hometeam_details = get_team_details(page_content_str,re_pattern)

    hometeam_coach = get_table_coach(hometeam_details)
    hometeam_lingdui = get_table_lingdui(hometeam_details)
    
    hometeam_table_header_lst = get_table_headers(hometeam_details)
    hometeam_table_cells_lst = get_table_cells(hometeam_details,len(hometeam_table_header_lst))

    df_hometeam_details = pd.DataFrame(data = hometeam_table_cells_lst,columns=hometeam_table_header_lst)
    return df_hometeam_details

###  客队统计数据

In [38]:
def get_awayteam_stats(page_content_str,re_pattern=
                       '<div class="part part02 blk">(.*?)<div class="part part03 blk">'):
    awayteam_details = get_team_details(page_content_str,re_pattern)

    awayteam_coach = get_table_coach(awayteam_details)
    awayteam_lingdui = get_table_lingdui(awayteam_details)
    
    awayteam_table_header_lst = get_table_headers(awayteam_details)
    awayteam_table_cells_lst = get_table_cells(awayteam_details,len(awayteam_table_header_lst))

    df_awayteam_details = pd.DataFrame(data = awayteam_table_cells_lst,columns=awayteam_table_header_lst)
    return df_awayteam_details



###  各项最高 (Later)

In [33]:
highest_details = re.findall('<div class="part part01 blk">(.*?)<div class="part part02 blk">'
                            ,page_content_str)[0]



# Loop and Scrape

In [40]:
df_list = []
for index,row in df_schedule_full.iterrows():
    game_UID = row['UID']
    game_hteam_name = row['主队']
    game_ateam_name = row['客队']
    base_url = row['统计_link']
    page_content_str = get_page_content(base_url,encoding='GB2312')
    
    # Get home team stats
    df_home = get_hometeam_stats(page_content_str=page_content_str)
    df_home['Game_ID'] = game_UID
    df_home['球队'] = game_hteam_name
    df_home['对手'] = game_ateam_name
    df_list.append(df_home)
    
    # Get away team stats
    df_away = get_awayteam_stats(page_content_str=page_content_str)
    df_away['Game_ID'] = game_UID
    df_away['球队'] = game_ateam_name
    df_away['对手'] = game_hteam_name # use to get opponents stats
    df_list.append(df_away)
    print(row['UID'],' ',row['主队'],' ', row['客队'],' ',datetime.datetime.now())
    
    # pause for a few seconds to avoid getting banned
    time.sleep(np.random.rand()*10)

2019202000001   广东   辽宁   2020-01-18 11:24:30.081632
2019202000002   江苏   浙江   2020-01-18 11:24:41.167556
2019202000003   四川   新疆   2020-01-18 11:24:50.282240
2019202000004   山东   八一   2020-01-18 11:24:54.160462
2019202000005   北控   深圳   2020-01-18 11:25:08.957069
2019202000006   青岛   吉林   2020-01-18 11:25:18.680434
2019202000007   广厦   福建   2020-01-18 11:25:25.349110
2019202000008   上海   同曦   2020-01-18 11:25:31.653127
2019202000009   天津   北京   2020-01-18 11:25:40.010461
2019202000010   广州   山西   2020-01-18 11:25:48.700589
2019202000011   辽宁   广厦   2020-01-18 11:25:51.347722
2019202000012   北控   上海   2020-01-18 11:25:56.571975
2019202000013   同曦   江苏   2020-01-18 11:26:06.506585
2019202000014   山西   福建   2020-01-18 11:26:10.245976
2019202000015   青岛   八一   2020-01-18 11:26:13.462357
2019202000016   新疆   浙江   2020-01-18 11:26:17.243629
2019202000017   广东   四川   2020-01-18 11:26:21.882010
2019202000018   北京   山东   2020-01-18 11:26:23.315379
2019202000019   天津   吉林   2020-01-18 11:26:29.

2019202000156   广东   浙江   2020-01-18 11:40:45.964372
2019202000157   山东   山西   2020-01-18 11:40:49.929099
2019202000158   四川   江苏   2020-01-18 11:40:51.988849
2019202000159   深圳   天津   2020-01-18 11:40:55.277172
2019202000160   新疆   青岛   2020-01-18 11:41:07.908667
2019202000161   上海   北京   2020-01-18 11:41:15.471263
2019202000162   同曦   广州   2020-01-18 11:41:20.821284
2019202000163   浙江   广厦   2020-01-18 11:41:31.693693
2019202000164   辽宁   吉林   2020-01-18 11:41:36.320893
2019202000165   天津   江苏   2020-01-18 11:41:44.523774
2019202000166   北控   广东   2020-01-18 11:41:50.740580
2019202000167   深圳   福建   2020-01-18 11:41:51.789728
2019202000168   八一   四川   2020-01-18 11:42:00.589231
2019202000169   青岛   山西   2020-01-18 11:42:11.234545
2019202000170   新疆   山东   2020-01-18 11:42:15.468930
2019202000171   吉林   深圳   2020-01-18 11:42:19.508470
2019202000172   江苏   广东   2020-01-18 11:42:23.224833
2019202000173   四川   山东   2020-01-18 11:42:31.099140
2019202000174   广州   福建   2020-01-18 11:42:39.

2019202000311   四川   青岛   2020-01-18 11:58:34.041853
2019202000312   天津   八一   2020-01-18 11:58:41.402644
2019202000313   山东   江苏   2020-01-18 11:58:49.049276
2019202000314   北控   辽宁   2020-01-18 11:58:56.250628
2019202000315   上海   广州   2020-01-18 11:59:01.476461
2019202000316   浙江   广厦   2020-01-18 11:59:10.792221
2019202000317   新疆   深圳   2020-01-18 11:59:14.679272
2019202000318   北京   吉林   2020-01-18 11:59:21.537025
2019202000319   同曦   福建   2020-01-18 11:59:26.894638
2019202000320   广东   山西   2020-01-18 11:59:37.947465
2019202000321   天津   广州   2020-01-18 11:59:47.553565
2019202000322   山东   浙江   2020-01-18 11:59:56.561675
2019202000323   青岛   广厦   2020-01-18 11:59:59.099820
2019202000324   江苏   福建   2020-01-18 12:00:02.092278
2019202000325   北京   北控   2020-01-18 12:00:10.182596
2019202000326   辽宁   山西   2020-01-18 12:00:13.075641
2019202000327   八一   新疆   2020-01-18 12:00:22.672072
2019202000328   吉林   四川   2020-01-18 12:00:29.965416
2019202000329   广东   同曦   2020-01-18 12:00:37.

In [37]:
games_stats = pd.concat(df_list,ignore_index=True)

games_stats

,球员,号码,出场时间,首发,2分中-投,3分中-投,罚球中-投,进攻篮板,防守篮板,助攻,...,抢断,失误,盖帽,扣篮,被侵,快攻,得分,Game_ID,球队,对手
0,布鲁克斯,6,36,1,8-10(80%),2-10(20%),5-5(100%),0,3,9,...,2,1,0,1,5,0/0,27,2019202000001,广东,辽宁
1,任骏飞,20,30,1,5-8(63%),3-6(50%),0-0(0%),1,1,1,...,3,0,0,1,1,0/0,19,2019202000001,广东,辽宁
2,易建联,9,35,1,7-8(88%),0-0(0%),2-4(50%),1,11,2,...,1,1,2,5,3,0/0,16,2019202000001,广东,辽宁
3,赵睿,10,31,1,5-9(56%),0-3(0%),0-1(0%),2,5,4,...,2,0,0,0,4,0/0,10,2019202000001,广东,辽宁
4,胡明轩,3,13,1,0-1(0%),1-3(33%),0-0(0%),0,1,0,...,0,1,0,0,0,0/0,3,2019202000001,广东,辽宁
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9640,刁成灏,24,0,0,0-0(0%),0-0(0%),0-0(0%),0,0,0,...,0,0,0,0,0,0/0,0,2019202000260,福建,青岛
9641,孙椿棚,13,0,0,0-0(0%),0-0(0%),0-0(0%),0,0,0,...,0,0,0,0,0,0/0,0,2019202000260,福建,青岛
9642,黄毅超,14,0,0,0-0(0%),0-0(0%),0-0(0%),0,0,0,...,0,0,0,0,0,0/0,0,2019202000260,福建,青岛
9643,汤杰,7,0,0,0-0(0%),0-0(0%),0-0(0%),0,0,0,...,0,0,0,0,0,0/0,0,2019202000260,福建,青岛


In [38]:
# remove team stats
if 2*len(games_stats['Game_ID'].unique())==len(games_stats.loc[games_stats['号码']=='--']):
    games_stats.drop(games_stats.loc[games_stats['号码']=='--'].index,inplace=True)

def split_made_attempt(df_orig):
    df = df_orig.copy()
    for col_name in list(filter(lambda x: '-' in x, df.columns.tolist())):
        orig_col = col_name
        col_made = re.findall('(.*)中-投',col_name)[0]+'中'
        col_attempt = re.findall('(.*)中-投',col_name)[0]+'投'
        df[[col_made,col_attempt]] = df[orig_col].str.split('-',expand=True)
        df[col_attempt] = df[col_attempt].apply(lambda x: re.sub('[(].*[)]','',x))
        df.drop(columns=orig_col,inplace=True)
    return df

games_stats = split_made_attempt(games_stats)
games_stats

,球员,号码,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,抢断,失误,...,得分,Game_ID,球队,对手,2分中,2分投,3分中,3分投,罚球中,罚球投
0,布鲁克斯,6,36,1,0,3,9,3,2,1,...,27,2019202000001,广东,辽宁,8,10,2,10,5,5
1,任骏飞,20,30,1,1,1,1,3,3,0,...,19,2019202000001,广东,辽宁,5,8,3,6,0,0
2,易建联,9,35,1,1,11,2,2,1,1,...,16,2019202000001,广东,辽宁,7,8,0,0,2,4
3,赵睿,10,31,1,2,5,4,3,2,0,...,10,2019202000001,广东,辽宁,5,9,0,3,0,1
4,胡明轩,3,13,1,0,1,0,1,0,1,...,3,2019202000001,广东,辽宁,0,1,1,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9639,胡珑贸,10,3,0,0,0,0,0,0,1,...,0,2019202000260,福建,青岛,0,0,0,1,0,0
9640,刁成灏,24,0,0,0,0,0,0,0,0,...,0,2019202000260,福建,青岛,0,0,0,0,0,0
9641,孙椿棚,13,0,0,0,0,0,0,0,0,...,0,2019202000260,福建,青岛,0,0,0,0,0,0
9642,黄毅超,14,0,0,0,0,0,0,0,0,...,0,2019202000260,福建,青岛,0,0,0,0,0,0


In [39]:
games_stats.to_csv(f'StatsData/All_Games_Stats_{datetime.date.today()}.csv',encoding = 'UTF-8',
                   index=False)